<a href="https://colab.research.google.com/github/likithavummiti/LIKITHA-FMML-LABS/blob/main/copy_of_fmml_mod_1_lab_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In [ ]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-9-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


In [ ]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

In [ ]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

In [ ]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

In [ ]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

In [ ]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


In [ ]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

In [ ]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


In [ ]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


In [ ]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34048257372654156


In [ ]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [ ]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [ ]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.33584635395170215
test accuracy is  0.34917953667953666


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


1A  
Yes, averaging validation accuracy across multiple splits of your dataset can often give you more consistent and reliable results compared to relying on a single validation split. This practice is commonly known as "cross-validation." Cross-validation is particularly useful in machine learning and data analysis when you want to assess the performance of a model and reduce the impact of the randomness associated with a single train-test split.
Here's why averaging validation accuracy across multiple splits is beneficial:

Reduced Variance: A single train-test split can lead to variability in the validation results because it depends on the specific data points chosen for training and testing. Averaging results over multiple splits helps mitigate this variance and provides a more stable estimate of model performance.

Better Generalization Assessment: Cross-validation allows you to assess how well your model generalizes to different subsets of the data. By averaging results across multiple splits, you get a more comprehensive view of your model's ability to handle various data configurations.Enhanced Hyperparameter Tuning: When tuning hyperparameters, you can use cross-validation to test different parameter settings across multiple data splits. Averaging the results helps you select the settings that perform consistently well across various data subsets.

Common cross-validation techniques include k-fold cross-validation and stratified k-fold cross-validation:

K-Fold Cross-Validation: In k-fold cross-validation, the dataset is divided into k equally sized subsets (folds). The model is trained and tested k times, each time using a different fold as the test set and the remaining k-1 folds as the training set. The validation scores from each iteration are averaged to obtain the final performance estimate.Stratified K-Fold Cross-Validation: Stratified k-fold cross-validation ensures that each fold maintains the same class distribution as the original dataset. This is particularly useful when dealing with imbalanced datasets.

Leave-One-Out Cross-Validation (LOOCV): This is a special case of k-fold cross-validation where k is set to the number of samples in the dataset. It provides a more exhaustive assessment of model performance but can be computationally expensive for large datasets.


2A   To provide a more accurate estimate of test accuracy, it's important to use proper evaluation techniques when assessing the performance of a machine learning model. Here are some key considerations:

Cross-Validation: Cross-validation is a technique where the dataset is split into multiple subsets (folds), and the model is trained and evaluated multiple times, using different subsets for testing each time. This helps in obtaining a more reliable estimate of the model's performance, reducing the impact of data variability.

Holdout Testing: Keep a separate portion of your dataset as a holdout test set that you don't use during model training. After training your model on the training data, evaluate its performance on this holdout test set to get an estimate of how well it generalizes to unseen data.

Metrics: Choose appropriate evaluation metrics based on the nature of your problem. Common metrics for classification tasks include accuracy, precision, recall, F1-score, and ROC AUC. For regression tasks, metrics like Mean Absolute Error (MAE) and Root Mean Squared Error (RMSE) are commonly used.

Stratified Sampling: In classification tasks, ensure that your data is stratified when splitting into training and testing sets. This ensures that each class is represented proportionally in both the training and testing sets, reducing the risk of biased evaluations.

Cross-Validation Strategies: Depending on your dataset size and characteristics, choose an appropriate cross-validation strategy. Common methods include k-fold cross-validation, stratified k-fold cross-validation, and leave-one-out cross-validation.

Hyperparameter Tuning: Optimize the hyperparameters of your machine learning model using techniques like grid search or random search. This can lead to improvements in model performance.

Outliers and Data Preprocessing: Carefully handle outliers and preprocess your data to ensure that it meets the assumptions of your chosen machine learning algorithm.

Ensemble Methods: Consider using ensemble methods like bagging (e.g., Random Forests) or boosting (e.g., Gradient Boosting) to improve model performance and reduce overfitting.

In summary, obtaining a more accurate estimate of test accuracy requires a combination of proper data splitting, evaluation metrics, preprocessing, and model tuning. It's essential to follow best practices in machine learning to ensure that your model's performance assessment is as accurate as possible and that it generalizes well to unseen data.







3A   The effect of the number of iterations on an estimate can vary depending on the specific estimation method and the problem being solved. In general, increasing the number of iterations can lead to a more accurate estimate, but there are diminishing returns, and other factors may come into play. Here are some general considerations:

Convergence: In many iterative estimation algorithms, the estimate tends to approach a stable value as the number of iterations increases. This means that initially, increasing iterations can lead to a significant improvement in accuracy, but as you continue to increase iterations, the improvement becomes smaller and eventually plateaus. This behavior is often described as convergence.

Computational Cost: More iterations generally require more computational resources, such as time and processing power. There may be a trade-off between the accuracy of the estimate and the computational cost. You need to balance the computational resources available with the desired level of accuracy.

Precision vs. Accuracy: It's important to distinguish between precision and accuracy. Precision refers to how consistent the estimates are when repeated measurements are made under the same conditions. Accuracy, on the other hand, is how close the estimate is to the true value. Increasing iterations may improve precision (reduce variability) but not necessarily accuracy if the estimation method is biased or based on incorrect assumptions.

Convergence Criteria: Many iterative algorithms use convergence criteria to stop the iterations when a certain level of accuracy is achieved. Setting appropriate convergence criteria is essential to avoid unnecessary iterations and computational overhead.

Initial Guess: The choice of the initial guess or starting point can also influence the effect of iterations. A poor initial guess may require more iterations to converge to an accurate estimate.

In summary, increasing the number of iterations can lead to a better estimate, but the relationship between the number of iterations and the quality of the estimate is not necessarily linear, and there are trade-offs to consider, including computational cost. It's essential to understand the specific estimation method and the problem context to determine the optimal number of iterations for your needs. In some cases, additional iterations may provide only marginal improvements in accuracy, making it more efficient to stop earlier.







4A   Increasing the number of iterations may help mitigate the limitations of a small training dataset to some extent, but it's not a complete solution to the challenges posed by a small dataset. Here are some considerations:

Overfitting: If you have a very small training dataset, increasing the number of iterations may lead to overfitting. Overfitting occurs when a model learns to fit the noise in the training data, rather than the underlying patterns. This can result in poor generalization to new, unseen data. Therefore, it's crucial to strike a balance between the number of iterations and the capacity of your model. Regularization techniques and careful monitoring of validation performance can help prevent overfitting.

Data Quality: The quality of the training data is critical. If your small dataset contains errors, outliers, or biases, increasing iterations won't necessarily improve the quality of the model's estimates. Data preprocessing and cleaning are essential steps to address these issues.

Validation Dataset: While increasing iterations might improve the model's performance on the training data, it may not necessarily improve its performance on a small validation dataset. The validation dataset is crucial for assessing the model's ability to generalize to new data. If your validation dataset is small, it may be more prone to random variations, and the model's performance on it may not be a reliable indicator of generalization.

Model Complexity: The choice of model architecture is also essential. A very complex model may have a higher risk of overfitting a small dataset, even with many iterations. You may need to consider using simpler models or techniques like transfer learning, which leverage pre-trained models on larger datasets.

Data Augmentation: Data augmentation techniques can artificially increase the effective size of your dataset by creating new training examples from the existing ones. This can be particularly useful when working with small datasets.

Regularization: Regularization techniques like L1 or L2 regularization can help control overfitting when dealing with small datasets.

In summary, while increasing the number of iterations may help a model learn more from a small training dataset, it's just one aspect of addressing the challenges associated with limited data. To build robust and generalizable models with small datasets, you should consider a combination of techniques, including data preprocessing, data augmentation, regularization, and careful model selection, in addition to optimizing the number of iterations.




